In [ ]:
import os
import warnings
import pandas as pd
from openpyxl import load_workbook, Workbook
from openpyxl.styles import Font, Alignment, Border, Side, PatternFill
from openpyxl.utils.dataframe import dataframe_to_rows
from copy import copy
import requests
from bs4 import BeautifulSoup
import wget
import ssl
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

data_atual = datetime.now()
data_atual_f = data_atual.date().strftime('%d-%m-%Y')
data_ontem = data_atual - timedelta(days=1)
data_ontem_f = data_ontem.date().strftime('%d-%m-%Y')

ssl._create_default_https_context = ssl._create_unverified_context

try:
    url = f'https://extrator.sefaz.al.gov.br/DESPESAS/COMPARATIVO-DESPESAS/despesa_empenhado_liquidado_pago_2024_siafe_gerado_em_{data_ontem_f}.xlsx'
    arquivo = os.path.join('S:/SOP/003 - GERÊNCIA DE ESTUDOS E PROJEÇÕES/DASHBOARDS POWERBI/Base python', f'despesa_empenhado_liquidado_pago_2024_siafe_gerado_em_{data_ontem_f}.xlsx')
    arquivo_csv = os.path.join('S:/SOP/003 - GERÊNCIA DE ESTUDOS E PROJEÇÕES/DASHBOARDS POWERBI/Base python', f'despesa_empenhado_liquidado_pago_2024_siafe_gerado_em_{data_ontem_f}.csv')
    wget.download(url, arquivo)
except:
    url = f'https://extrator.sefaz.al.gov.br/DESPESAS/COMPARATIVO-DESPESAS/despesa_empenhado_liquidado_pago_2024_siafe_gerado_em_{data_atual_f}.xlsx'
    arquivo = os.path.join('S:/SOP/003 - GERÊNCIA DE ESTUDOS E PROJEÇÕES/DASHBOARDS POWERBI/Base python', f'despesa_empenhado_liquidado_pago_2024_siafe_gerado_em_{data_atual_f}.xlsx')
    arquivo_csv = os.path.join('S:/SOP/003 - GERÊNCIA DE ESTUDOS E PROJEÇÕES/DASHBOARDS POWERBI/Base python', f'despesa_empenhado_liquidado_pago_2024_siafe_gerado_em_{data_atual_f}.csv')
    wget.download(url, arquivo)

base_arquivo = os.path.join('S:/SOP/003 - GERÊNCIA DE ESTUDOS E PROJEÇÕES/SCRIPTS/Bases empenhado - Layne', f'Base Acompanhamento - atualizado {data_atual_f}.xlsx')


# Baixar o arquivo
wget.download(url, arquivo)
df = pd.read_excel(arquivo)

df_f = df[['DESCRICAO_UO', 'PT', 'PT_DESCRICAO', 'NATUREZA6', 'VALOR_EMPENHADO', 'MES']]
for i in ['DESCRICAO_UO', 'PT', 'PT_DESCRICAO', 'NATUREZA6', 'MES']:
    df_f[i] = df_f[i].astype(str)

# concatenando as colunas para o merge
df_x = df_f
df_x['concat'] = df_x['DESCRICAO_UO']
for i in ['PT', 'NATUREZA6']:
    df_x['concat'] = df_x['concat'] + df_x[i]

# base com as informacoes
info = pd.read_excel('S:/SOP/003 - GERÊNCIA DE ESTUDOS E PROJEÇÕES/DASHBOARDS POWERBI/Base python/Base acompanhamento Layne.xlsx')

# fazendo o merge para cada valor e cada mês

for i in df_f['MES'].unique():
    info[i] = ''
    teste = df_x.loc[df_x['MES'] == i]
    for j in info['concat'].values:
     info.loc[info.concat == j, i] = teste.loc[teste.concat == j]['VALOR_EMPENHADO'].sum()

# Dropando o concat

info.drop(columns='concat', inplace=True)

# Criando a coluna de total dos meses contidos

info['TOTAL'] = info.iloc[ : , 4:].sum(axis=1)

# Padronizando as colunas e organiznado

info.rename(columns={'DESCRICAO_UO': 'UO', 'PT': 'PROGRAMA DE TRABALHO',
                      'PT_DESCRICAO': 'PROGRAMA DE TRABALHO DESCRIÇÃO',
                        'NATUREZA6': 'NATUREZA'}, inplace=True)

# Padronizando os meses

meses_padronizados = {
    '1': 'JAN', '2': 'FEV', '3': 'MAR', '4': 'ABR',
    '5': 'MAI', '6': 'JUN', '7': 'JUL', '8': 'AGO',
    '9': 'SET', '10': 'OUT', '11': 'NOV', '12': 'DEZ'
}

info.rename(columns=meses_padronizados, inplace=True)

# Padronizando as naturezas

def formatar_natureza(x):
    x_str = str(x)
    x_mod = x_str[1:]
    x_mod = x_str[:-2]
    return int(x_mod)

info['NATUREZA'] = info['NATUREZA'].apply(formatar_natureza)

# Padronizando os PTs

pt_padronizados = {'10302101550690000': '10.302. 1015. 5069',
                   '6122000420010000': '06.122. 0004. 2001',
                   '20605104335800000': '20.605. 1043. 3580',
                   '8306104352110000': '08.306. 1043. 5211',
                   '12368101237620000': '12.368. 1012. 3762',
                   '12368101237620000': '12.368. 1012. 3762',
                   '6181101738370000': '06.181. 1017. 3837',
                   '14122101751890000': '14.122. 1017. 5189',
                   '14122101751890000': '14.122. 1017. 5189',
                   '14122101751890000': '14.122. 1017. 5189',
                   '14122101751890000': '14.122. 1017. 5189',
                   '14421000451850000': '14.421. 0004. 5185',
                   '14421000451850000': '14.421. 0004. 5185',
                   '14421000451850000': '14.421. 0004. 5185',
                   '6122101952750000': '06.122. 1019. 5275',
                   '6122101952750000': '06.122. 1019. 5275'}

info['PROGRAMA DE TRABALHO'] = info['PROGRAMA DE TRABALHO'].astype(str)
info['PROGRAMA DE TRABALHO'] = info['PROGRAMA DE TRABALHO'].map(pt_padronizados)

# Salvando

info.to_excel(base_arquivo, index=False)

modelo_wb = load_workbook('S:/SOP/003 - GERÊNCIA DE ESTUDOS E PROJEÇÕES/SCRIPTS/Bases empenhado - Layne/Base Acompanhamento - Modelo.xlsx')
modelo_ws = modelo_wb.active

novo_wb = Workbook()
novo_ws = novo_wb.active

# Transferência de dados e formatação do df
for r_idx, row in enumerate(dataframe_to_rows(info, index=False, header=True), 1):
    for c_idx, value in enumerate(row, 1):
        cell = novo_ws.cell(row=r_idx, column=c_idx, value=value)
        modelo_cell = modelo_ws.cell(row=r_idx, column=c_idx)
        if modelo_cell.has_style:
            cell.font = copy(modelo_cell.font)
            cell.border = copy(modelo_cell.border)
            cell.fill = copy(modelo_cell.fill)
            cell.number_format = copy(modelo_cell.number_format)
            cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)

# Ajustar largura das colunas
for col in novo_ws.columns:
    max_length = 0
    column = col[0].column_letter
    for cell in col:
        if cell.value and len(str(cell.value)) > max_length:
            max_length = len(str(cell.value))
    adjusted_width = max_length + 2
    novo_ws.column_dimensions[column].width = adjusted_width

# Definir largura específica para a coluna A e D e centralizar todo o conteúdo
novo_ws.column_dimensions['A'].width = 40
novo_ws.column_dimensions['C'].width = 11
novo_ws.column_dimensions['D'].width = 60

# Ajuste específico para as colunas E até J
for col in ['E', 'F', 'G', 'H', 'I', 'J']:
    novo_ws.column_dimensions[col].width = 20 

# Mesclar células de A18 a D18
novo_ws.merge_cells('A18:D18')

# Formatar célula mesclada A18 a D18 e coluna J conforme a célula modelo
modelo_cell_A18 = modelo_ws['A18']
novo_ws.merge_cells('A18:D18')
merged_cell = novo_ws['A18']
merged_cell.value = "TOTAL"
merged_cell.font = Font(bold=True)
merged_cell.font = copy(modelo_cell_A18.font)
merged_cell.border = copy(modelo_cell_A18.border)
merged_cell.fill = copy(modelo_cell_A18.fill)
merged_cell.alignment = copy(modelo_cell_A18.alignment)

# Inserir fórmula de soma na célula J18 e aplicar formatação do modelo

novo_ws['J18'] = f"=SUM(J2:J17)"
modelo_cell_I1 = modelo_ws['I1']
modelo_cell_I2 = modelo_ws['I2']
novo_ws['J1'].font = copy(modelo_cell_I1.font)
novo_ws['J1'].border = copy(modelo_cell_I1.border)
novo_ws['J1'].fill = copy(modelo_cell_I1.fill)
novo_ws['J1'].number_format = copy(modelo_cell_I1.number_format)
novo_ws['J1'].alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)


for c in range(5, 11):
    for i in range(2, 19):
        novo_ws.cell(row=i, column=c).font = copy(modelo_cell_I2.font)
        novo_ws.cell(row=i, column=c).border = copy(modelo_cell_I2.border)
        novo_ws.cell(row=i, column=c).fill = copy(modelo_cell_I2.fill)
        novo_ws.cell(row=i, column=c).number_format = copy(modelo_cell_I2.number_format)
        novo_ws.cell(row=i, column=c).alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)

# Inserir fórmula de soma nas células E18 a I18
for col in range(5, 11):
    cell = novo_ws.cell(row=18, column=col)
    cell.value = f"=SUM({cell.column_letter}2:{cell.column_letter}17)"

novo_wb.save(base_arquivo)

info